### Tạo ra các đặc trưng mới cho dữ liệu

In [1]:
import pandas as pd
import numpy as np

### 1. Load dữ liệu

In [2]:
# Load dữ liệu thô đã thu thập cho từng đội bóng
arsenal_df = pd.read_csv("../../data/arsenal/raw.csv")

In [3]:
arsenal_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,captain,formation,referee,match report,notes,sot,pkatt,season,team,url
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2,0,Crystal Palace,...,Martin Ødegaard,4-3-3,Anthony Taylor,Match Report,NaN,2.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,2,Leicester City,...,Martin Ødegaard,4-3-3,Darren England,Match Report,NaN,7.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3,0,Bournemouth,...,Martin Ødegaard,4-3-3,Craig Pawson,Match Report,NaN,6.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2,1,Fulham,...,Martin Ødegaard,4-3-3,Jarred Gillett,Match Report,NaN,8.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2,1,Aston Villa,...,Martin Ødegaard,4-3-3,Robert Jones,Match Report,NaN,8.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...


### 2. Bổ sung thêm các đặc trưng:
- Phong độ hiện tại (%W, %L, %D) của đội đang đi xây dựng model trong 5 trận gần nhất
- Lịch sử đối đầu (%W, %D) của đội đang đi xây dựng model với đối thủ
- Đội đối thủ có phải là big 6 hay không?

In [4]:
# Tạo cột date_time để sắp xếp các trận tăng dần theo thời gian
arsenal_df["time"].fillna("00:00", inplace=True)
arsenal_df["date_time"] = pd.to_datetime(arsenal_df["date"] + " " + arsenal_df["time"])
arsenal_df = arsenal_df.sort_values(by="date_time")

In [5]:
arsenal_df.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,formation,referee,match report,notes,sot,pkatt,season,team,url,date_time
342,2013-08-17,00:00,Premier League,Matchweek 1,Sat,Home,L,1,3,Aston Villa,...,4-2-3-1,Anthony Taylor,Match Report,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-08-17
343,2013-08-24,00:00,Premier League,Matchweek 2,Sat,Away,W,3,1,Fulham,...,4-2-3-1,Howard Webb,Match Report,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-08-24
344,2013-09-01,00:00,Premier League,Matchweek 3,Sun,Home,W,1,0,Tottenham,...,4-2-3-1,Michael Oliver,Match Report,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-09-01
345,2013-09-14,00:00,Premier League,Matchweek 4,Sat,Away,W,3,1,Sunderland,...,4-2-3-1,Martin Atkinson,Match Report,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-09-14
346,2013-09-22,00:00,Premier League,Matchweek 5,Sun,Home,W,3,1,Stoke City,...,4-2-3-1,Mike Dean,Match Report,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-09-22


I. Tính toán phong độ của đội đang đi xây dựng model trong 5 trận gần nhất

In [9]:
arsenal_df["win_before"] = 0.0
arsenal_df["lose_before"] = 0.0
arsenal_df["draw_before"] = 0.0

for index, row in arsenal_df.iterrows():
    date_time = row["date_time"]
    season = row["season"]

    # Lấy các trận đấu trước đó của đội bóng
    previous_matches = arsenal_df[
        (arsenal_df["date_time"] < date_time)
        & (arsenal_df["season"] == season)
    ]

    # Chỉ lấy 5 trận gần nhất
    previous_matches = previous_matches[-5:]

    # Đếm số trận thắng, hoà trước đó
    win_before = previous_matches[previous_matches["result"] == "W"].shape[0]
    lose_before = previous_matches[previous_matches["result"] == "L"].shape[0]
    draw_before = previous_matches[previous_matches["result"] == "D"].shape[0]

    # Tính tỉ lệ số trận thắng, hoà trước đó
    match_total = 1 if previous_matches.shape[0] == 0 else previous_matches.shape[0]  
    arsenal_df.at[index, "win_before"] = win_before / match_total
    arsenal_df.at[index, "lose_before"] = lose_before / match_total
    arsenal_df.at[index, "draw_before"] = draw_before / match_total

In [11]:
arsenal_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,notes,sot,pkatt,season,team,url,date_time,win_before,draw_before,lose_before
342,2013-08-17,00:00,Premier League,Matchweek 1,Sat,Home,L,1,3,Aston Villa,...,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-08-17 00:00:00,0.000000,0.0,0.000000
343,2013-08-24,00:00,Premier League,Matchweek 2,Sat,Away,W,3,1,Fulham,...,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-08-24 00:00:00,0.000000,0.0,1.000000
344,2013-09-01,00:00,Premier League,Matchweek 3,Sun,Home,W,1,0,Tottenham,...,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-09-01 00:00:00,0.500000,0.0,0.500000
345,2013-09-14,00:00,Premier League,Matchweek 4,Sat,Away,W,3,1,Sunderland,...,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-09-14 00:00:00,0.666667,0.0,0.333333
346,2013-09-22,00:00,Premier League,Matchweek 5,Sun,Home,W,3,1,Stoke City,...,NaN,NaN,NaN,2013-2014,Arsenal,https://fbref.com/en/squads/18bb7c10/2013-2014...,2013-09-22 00:00:00,0.750000,0.0,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023-05-02,20:00,Premier League,Matchweek 34,Tue,Home,W,3,1,Chelsea,...,NaN,9.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...,2023-05-02 20:00:00,0.200000,0.6,0.200000
34,2023-05-07,16:30,Premier League,Matchweek 35,Sun,Away,W,2,0,Newcastle Utd,...,NaN,6.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...,2023-05-07 16:30:00,0.200000,0.6,0.200000
35,2023-05-14,16:30,Premier League,Matchweek 36,Sun,Home,L,0,3,Brighton,...,NaN,2.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...,2023-05-14 16:30:00,0.400000,0.4,0.200000
36,2023-05-20,17:30,Premier League,Matchweek 37,Sat,Away,L,0,1,Nott'ham Forest,...,NaN,3.0,0.0,2022-2023,Arsenal,https://fbref.com/en/squads/18bb7c10/2022-2023...,2023-05-20 17:30:00,0.400000,0.2,0.400000


II. Tính toán lịch sử sử đối đầu giữa 2 đội